In [1]:
import pandas as pd
import numpy as np

In [2]:
dt = pd.read_csv('../DATA/healthcare-dataset-stroke-data.csv')

In [3]:
dt.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
dt.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [5]:
dt['bmi']= dt['bmi'].replace(np.nan, dt['bmi'].median())

In [6]:
cat_cols = ['gender','ever_married','work_type','Residence_type','smoking_status']

In [7]:
from sklearn.preprocessing import OneHotEncoder 

In [8]:
onehotencoder = OneHotEncoder(sparse=False, categories='auto')

In [9]:
transformed_data = onehotencoder.fit_transform(dt[cat_cols])

In [10]:
cat_columns = ['gender_female','gender_male','gender_other','ever_married_no','ever_married_yes','work_type_govt_job','work_type_never_worked',
               'work_type_private','work_type_self_employed','work_type_children','resident_type_rural','resident_type_urban','smoking_status_unknown',
               'smoking_status_formerly_smoked','smoking_status_never_smoked','smoking_status_smokes']

In [11]:
transformed_data = pd.DataFrame(transformed_data,index=dt.index,columns = cat_columns)

In [12]:
transformed_data.sum()

gender_female                     2994.0
gender_male                       2115.0
gender_other                         1.0
ever_married_no                   1757.0
ever_married_yes                  3353.0
work_type_govt_job                 657.0
work_type_never_worked              22.0
work_type_private                 2925.0
work_type_self_employed            819.0
work_type_children                 687.0
resident_type_rural               2514.0
resident_type_urban               2596.0
smoking_status_unknown            1544.0
smoking_status_formerly_smoked     885.0
smoking_status_never_smoked       1892.0
smoking_status_smokes              789.0
dtype: float64

In [13]:
X = pd.concat([dt[['age','bmi','avg_glucose_level']],transformed_data],axis = 1)

In [14]:
X.columns

Index(['age', 'bmi', 'avg_glucose_level', 'gender_female', 'gender_male',
       'gender_other', 'ever_married_no', 'ever_married_yes',
       'work_type_govt_job', 'work_type_never_worked', 'work_type_private',
       'work_type_self_employed', 'work_type_children', 'resident_type_rural',
       'resident_type_urban', 'smoking_status_unknown',
       'smoking_status_formerly_smoked', 'smoking_status_never_smoked',
       'smoking_status_smokes'],
      dtype='object')

In [15]:
y = dt[['stroke']]

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=3)

In [17]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [23]:
y_res['stroke'].value_counts()

1    3884
0    3884
Name: stroke, dtype: int64

In [18]:
#Applying decision tree model using hyper parameter optimization using grid serach CV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Number of trees in random forest
n_estimators = [400,500,600]
# Number of features to consider at every split
max_features = [6,8,10]
# Maximum number of levels in tree
max_depth = [4,6,8,10]
# Minimum number of samples required to split a node
min_samples_split = [10,20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [10,20]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [19]:
# Instantiating Decision Tree classifier 
rf = RandomForestClassifier() 
  
# Instantiating GridSearchCV object 
rf_cv = GridSearchCV(rf, param_dist, cv = 3)

In [24]:
#fitting X and y
rf_cv.fit(X_res, y_res)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was pa

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was pa

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was pa

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was pa

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was pa

...

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was pa

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was pa

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was pa

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was pa

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False], 'max_depth': [4, 6, 8, 10],
                         'max_features': [6, 8, 10],
                         'min_samples_leaf': [10, 20],
                         'min_samples_split': [10, 20],
                         'n_estimators': [400, 500, 600]})

In [25]:
# Print the tuned parameters and score 
print("Tuned Random Forest Parameters: {}".format(rf_cv.best_params_)) 
print("Best score is {}".format(rf_cv.best_score_))

Tuned Random Forest Parameters: {'bootstrap': False, 'max_depth': 10, 'max_features': 6, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 400}
Best score is 0.9639604345282212


In [26]:
predictions_rf = rf_cv.predict(X_test)

In [27]:
from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(y_test,predictions_rf))

#print(confusion_matrix(y_test,predictions_rf))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97       977
           1       0.17      0.04      0.07        45

    accuracy                           0.95      1022
   macro avg       0.56      0.52      0.52      1022
weighted avg       0.92      0.95      0.93      1022



In [28]:
predictions_rf_train = rf_cv.predict(X_train)

In [29]:
print(classification_report(y_train,predictions_rf_train))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      3884
           1       0.70      0.21      0.32       204

    accuracy                           0.96      4088
   macro avg       0.83      0.60      0.65      4088
weighted avg       0.95      0.96      0.94      4088



In [30]:
# loading library
import pickle

# create an iterator object with write permission - model.pkl
with open('../models/strokes/stroke_model.pkl', 'wb') as files:
    pickle.dump(rf_cv, files)
    
with open('../models/strokes/one_hot_strokes.pkl', 'wb') as files:
    pickle.dump(onehotencoder, files)

In [31]:
import pickle
ml_model = pickle.load(open('../models/strokes/stroke_model.pkl', 'rb'))
one_hot_model = pickle.load(open('../models/strokes/one_hot_strokes.pkl', 'rb'))

In [32]:
single_observation = dt.loc[[0]]

In [33]:
cat_data = one_hot_model.transform(single_observation[['gender','ever_married','work_type','Residence_type','smoking_status']])

In [40]:
cat_data

,gender_female,gender_male,gender_other,ever_married_no,ever_married_yes,work_type_govt_job,work_type_never_worked,work_type_private,work_type_self_employed,work_type_children,resident_type_rural,resident_type_urban,smoking_status_unknown,smoking_status_formerly_smoked,smoking_status_never_smoked,smoking_status_smokes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [34]:
cat_data = pd.DataFrame(cat_data,columns = cat_columns)

In [35]:
X = pd.concat([single_observation[['age','bmi','avg_glucose_level']],cat_data],axis = 1)

In [36]:
single_predictions = ml_model.predict(X)

In [38]:
single_predictions[0]

0